In [4]:
import json
import numpy as np
import requests
from tqdm import tqdm

# Klient eksperymentu AB
- klient eksperymentu AB służy do przetestowania, czy:
    - działają predykcje wytrenowanego modelu oraz modelu bazowego
    - działa serwer z REST API wraz z testami AB
    - działa eksperyment AB
- Wykonano następujące działania w poniższym notatniku:
    - zresetowanie bazy konwersji w serwerze
    - inicjalizacja liczby przetwarzanych wydarzeń - 10000
    - pętla generowania wydarzeń i zapytań o predykcje
    - podsumowanie testu AB
- Przed uruchomieniem notatnika należy uruchomić serwer (instrukcja w [README.md](../README.md))

In [5]:
url = "http://127.0.0.1:8000/"
# /predict POST -> all 30 attributes
# /conversion POST -> user_id, value
# /resultsAB GET -> get all conversions in JSON
# /resetAB POST -> reset memory of conversions

headers = {
    'Content-Type': 'application/json'
}


### Zresetowanie bazy konwersji w serwerze

In [6]:
reset_response = requests.request("POST", url+'resetAB/')

### Inicjalizacja liczby przetwarzanych wydarzeń - 10000

In [7]:
N = 10000
print("Processing",N,"events...")

Processing 10000 events...


### Pętla generowania wydarzeń i zapytań o predykcje
- pętla generuje zadaną liczbę wydarzeń.
- na początku występuje generator atrybutów, który z możliwie prostym rozkładem (normalnym lub jednostajnym) generuje losowe atrybuty o podobnym rzędzie wielkości do atrybutów z danych wejściowych
- wysyłane są zapytania do serwera z listą atrybutów z prośbą o predykcję optymalnej zniżki
- wykonywany jest test AB:
    - id użytkownika to po prostu wartość zmiennej i - numer itercji pętli - dzięki temu jest równo połowa użytkowników o parzystych numerach id i druga połowa o nieparzystych numerach id. Są więc spełnione warunki eksperymentu AB.
    - dla parzystych identyfikatorów użytkownika (funkcja mieszająca to id % 2) wykonywana jest predykcja z modelu "dummy" (modelu bazowego), który z rozkładem jednostajnym losuje prawdopodobieństwo, że klient kupi przy danej zniżce. Po analizie wszystkich zniżek serwer zwraca najlepszą zniżkę.
    - dla nieparzystych identyfikatorów użytkownika wykonywane są predykcje za pomocą modelu "tuned", wytrenowanego przez nas w [tym notatniku](../4-train/train.ipynb).
    - klient pobiera zniżkę i z wprost proporcjonalnym prawdopodobieństwem do wysokości zniżki klient generuje decyzję o zakupie produktu lub nie
    - jeśli klient kupił produkt, wysyła informację do serswera o tzw. konwersji, czyli zysku.
    - model zapisuje wyniki

In [8]:
for i in tqdm(range(N)):
    # generator of attributes
    attributes = dict()
    attributes['user_id'] = i
    attributes['price'] = round(np.random.normal(2000, 1000),0)
    attributes['product_views_count'] = int(abs(round(np.random.normal(0, 15), 0)))
    attributes['category_view_count'] = int(abs(round(np.random.normal(0, 11), 0)))
    attributes['bought_with_lower_count'] = int(abs(round(np.random.normal(0, 6), 0)))
    attributes['bought_category_count'] = int(abs(round(np.random.normal(0, 7), 0)))
    attributes['time'] = abs(round(np.random.normal(0, 1000), 2))
    city_index = int(round(np.random.uniform(0, 7), 0))
    for j in range(8):
        attributes['one_hot_city' + str(j)] = 1 if j == city_index else 0

    category_index = round(np.random.uniform(0, 14))
    for j in range(15):
        attributes['one_hot_category' + str(j)] = 1 if j == category_index else 0

    # send request for predictions of discount
    json_data = json.dumps(attributes, ensure_ascii=True)
    response = requests.request("POST", url+'predict/', headers=headers, data=json_data)

    #%%

    discount = response.json()['discount']

    #%%

    # generate decision about buying
    import random
    rand_num = random.random()
    probability = (discount/100.0 + 0.1)
    buying = (rand_num < probability)
    # print("rand_num: ", rand_num)
    # print()
    # print("Decision ", i, " = ",buying)


    # if buying then post conversion to server
    if not buying:
        continue
    conversion_dictionary = dict()
    conversion_dictionary['user_id'] = i
    conversion_dictionary['value'] = float(attributes['price']*(40-discount)/100)
    conversion_data = json.dumps(conversion_dictionary)
    conversion_response = requests.request("POST", url+'conversion/', headers=headers, data=conversion_data)
    # print(conversion_response.content)

100%|██████████| 10000/10000 [04:07<00:00, 40.44it/s]


### Podsumowanie testu AB
- po wykonaniu wszystkich wydarzeń notatnik przechodzi do ewaluacji testu AB
- nie jest to już część klienta systemu, ale dla wygody jest umieszczona w tym samym notatniku
- pobierane są konwersje z serwera
- zliczana jest liczba zakupów i wartość zysku zależnie od wybranego modelu
- prezentowane są wyniki testu

In [9]:
# after all actions

# get all conversions in JSON
all_conversions = requests.request("GET", url+'resultsAB/')
conversions = all_conversions.json()

# count conversions and print results
value0 = 0
value1 = 0

count0 = 0
count1 = 0
for conversion in conversions:
    if int(conversion['model_id']) == 0:
        value0 += float(conversion['value'])
        count0 += 1
    else:
        value1 += float(conversion['value'])
        count1 += 1
print("Dummy conversion number: ", count0)
print("Tuned conversion number: ", count1)

print('\n')

print("Dummy conversion value: ", round(value0, 2))
print("Tuned conversion value: ", round(value1, 2))

Dummy conversion number:  725
Tuned conversion number:  1108


Dummy conversion value:  468982.35
Tuned conversion value:  581949.7
